(magnetic-resonance-imaging)=
# Magnetic Resonance Imaging

In [1]:
from myst_nb import glue
from IPython.display import display, Markdown, HTML
from schemacode import render, schema, utils
from markdown import markdown

# Load the schema path
schemapath = utils.get_schema_path()
schema_obj = schema.load_schema(schemapath)

(magnetic-resonance-imaging:common-metadata-fields)=
## Common metadata fields

MR Data described in the following sections share the following RECOMMENDED metadata
fields (stored in sidecar JSON files).
MRI acquisition parameters are divided into several categories based on
"A checklist for fMRI acquisition methods reporting in the literature"
([article](https://winnower-production.s3.amazonaws.com/papers/977/v4/pdf/977-a-checklist-for-fmri-acquisition-methods-reporting-in-the-literature.pdf),
[checklist](https://winnower-production.s3.amazonaws.com/papers/977/assets/993e199d-6bc3-4418-be3a-f620af1188b7-Parameter_Reporting_V1p3.pdf))
by Ben Inglis.

(magnetic-resonance-imaging:common-metadata-fields:scanner-hardware)=
### Scanner Hardware

{term}`Manufacturer (metadata)`

In [2]:
display(Markdown("{term}`Manufacturer (metadata)`"))

{term}`Manufacturer (metadata)`

In [3]:
display("{term}`Manufacturer (metadata)`")

'{term}`Manufacturer (metadata)`'

In [4]:
field_info = {
   "Manufacturer": ("RECOMMENDED", "Corresponds to DICOM Tag 0008, 0070 `Manufacturer`."),
   "ManufacturersModelName": ("RECOMMENDED", "Corresponds to DICOM Tag 0008, 1090 `Manufacturers Model Name`."),
   "DeviceSerialNumber": ("RECOMMENDED", "Corresponds to DICOM Tag 0018, 1000 `DeviceSerialNumber`."),
   "StationName": ("RECOMMENDED", "Corresponds to DICOM Tag 0008, 1010 `Station Name`."),
   "SoftwareVersions": ("RECOMMENDED", "Corresponds to DICOM Tag 0018, 1020 `Software Versions`."),
   "HardcopyDeviceSoftwareVersion": "DEPRECATED",
   "MagneticFieldStrength": "RECOMMENDED, but REQUIRED for Arterial Spin Labeling",
   "ReceiveCoilName": "RECOMMENDED",
   "ReceiveCoilActiveElements": (
      "RECOMMENDED",
      "See an example below the table.",
   ),
   "GradientSetType": "RECOMMENDED",
   "MRTransmitCoilSequence": "RECOMMENDED",
   "MatrixCoilMode": "RECOMMENDED",
   "CoilCombinationMethod": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(utils.fix(markdown(table.to_markdown(), extensions=['tables']))))

Key name,Requirement Level,Data type,Description
{term}`Manufacturer (metadata)`,RECOMMENDED,string,"Manufacturer of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 0070 Manufacturer."
{term}`ManufacturersModelName (metadata)`,RECOMMENDED,string,"Manufacturer's model name of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 1090 Manufacturers Model Name."
{term}`DeviceSerialNumber (metadata)`,RECOMMENDED,string,"The serial number of the equipment that produced the measurements. A pseudonym can also be used to prevent the equipment from being identifiable, so long as each pseudonym is unique within the dataset. Corresponds to DICOM Tag 0018, 1000 DeviceSerialNumber."
{term}`StationName (metadata)`,RECOMMENDED,string,"Institution defined name of the machine that produced the measurements. Corresponds to DICOM Tag 0008, 1010 Station Name."
{term}`SoftwareVersions (metadata)`,RECOMMENDED,string,"Manufacturer's designation of software version of the equipment that produced the measurements. Corresponds to DICOM Tag 0018, 1020 Software Versions."
{term}`HardcopyDeviceSoftwareVersion (metadata)`,DEPRECATED,string,"Manufacturer's designation of the software of the device that created this Hardcopy Image (the printer). Corresponds to DICOM Tag 0018, 101A Hardcopy Device Software Version."
{term}`MagneticFieldStrength (metadata)`,"RECOMMENDED, but REQUIRED for Arterial Spin Labeling",number,"Nominal field strength of MR magnet in Tesla. Corresponds to DICOM Tag 0018, 0087 Magnetic Field Strength."
{term}`ReceiveCoilName (metadata)`,RECOMMENDED,string,"Information describing the receiver coil. Corresponds to DICOM Tag 0018, 1250 Receive Coil Name, although not all vendors populate that DICOM Tag, in which case this field can be derived from an appropriate private DICOM field."
{term}`ReceiveCoilActiveElements (metadata)`,RECOMMENDED,string,Information describing the active/selected elements of the receiver coil. This does not correspond to a tag in the DICOM ontology. The vendor-defined terminology for active coil elements can go in this field. See an example below the table.
{term}`GradientSetType (metadata)`,RECOMMENDED,string,"It should be possible to infer the gradient coil from the scanner model. If not, for example because of a custom upgrade or use of a gradient insert set, then the specifications of the actual gradient coil should be reported independently."


In [5]:
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}Manufacturer (metadata),RECOMMENDED,string,"Manufacturer of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 0070 Manufacturer."
{term}ManufacturersModelName (metadata),RECOMMENDED,string,"Manufacturer's model name of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 1090 Manufacturers Model Name."
{term}DeviceSerialNumber (metadata),RECOMMENDED,string,"The serial number of the equipment that produced the measurements. A pseudonym can also be used to prevent the equipment from being identifiable, so long as each pseudonym is unique within the dataset. Corresponds to DICOM Tag 0018, 1000 DeviceSerialNumber."
{term}StationName (metadata),RECOMMENDED,string,"Institution defined name of the machine that produced the measurements. Corresponds to DICOM Tag 0008, 1010 Station Name."
{term}SoftwareVersions (metadata),RECOMMENDED,string,"Manufacturer's designation of software version of the equipment that produced the measurements. Corresponds to DICOM Tag 0018, 1020 Software Versions."
{term}HardcopyDeviceSoftwareVersion (metadata),DEPRECATED,string,"Manufacturer's designation of the software of the device that created this Hardcopy Image (the printer). Corresponds to DICOM Tag 0018, 101A Hardcopy Device Software Version."
{term}MagneticFieldStrength (metadata),"RECOMMENDED, but REQUIRED for Arterial Spin Labeling",number,"Nominal field strength of MR magnet in Tesla. Corresponds to DICOM Tag 0018, 0087 Magnetic Field Strength."
{term}ReceiveCoilName (metadata),RECOMMENDED,string,"Information describing the receiver coil. Corresponds to DICOM Tag 0018, 1250 Receive Coil Name, although not all vendors populate that DICOM Tag, in which case this field can be derived from an appropriate private DICOM field."
{term}ReceiveCoilActiveElements (metadata),RECOMMENDED,string,Information describing the active/selected elements of the receiver coil. This does not correspond to a tag in the DICOM ontology. The vendor-defined terminology for active coil elements can go in this field. See an example below the table.
{term}GradientSetType (metadata),RECOMMENDED,string,"It should be possible to infer the gradient coil from the scanner model. If not, for example because of a custom upgrade or use of a gradient insert set, then the specifications of the actual gradient coil should be reported independently."


Example for `ReceiveCoilActiveElements`:

For Siemens, coil channels are typically not activated/selected individually,
but rather in pre-defined selectable "groups" of individual channels,
and the list of the "groups" of elements that are active/selected in any
given scan populates the `Coil String` entry in Siemens' private DICOM fields
(for example, `HEA;HEP` for the Siemens standard 32 ch coil
when both the anterior and posterior groups are activated).
This is a flexible field that can be used as most appropriate for a given
vendor and coil to define the "active" coil elements.
Since individual scans can sometimes not have the intended coil elements selected,
it is preferable for this field to be populated directly from the DICOM
for each individual scan, so that it can be used as a mechanism for checking
that a given scan was collected with the intended coil elements selected

(magnetic-resonance-imaging:common-metadata-fields:sequence-specifics)=
### Sequence Specifics

In [6]:
field_info = {
   "PulseSequenceType": "RECOMMENDED",
   "ScanningSequence": "RECOMMENDED",
   "SequenceVariant": "RECOMMENDED",
   "ScanOptions": "RECOMMENDED",
   "SequenceName": "RECOMMENDED",
   "PulseSequenceDetails": "RECOMMENDED",
   "NonlinearGradientCorrection": "RECOMMENDED, but REQUIRED if [PET](./09-positron-emission-tomography.md) data are present",
   "MRAcquisitionType": "RECOMMENDED, but REQUIRED for Arterial Spin Labeling",
   "MTState": "RECOMMENDED",
   "MTOffsetFrequency": "RECOMMENDED if the MTstate is `True`.",
   "MTPulseBandwidth": "RECOMMENDED if the MTstate is `True`.",
   "MTNumberOfPulses": "RECOMMENDED if the MTstate is `True`.",
   "MTPulseShape": "RECOMMENDED if the MTstate is `True`.",
   "MTPulseDuration": "RECOMMENDED if the MTstate is `True`.",
   "SpoilingState": "RECOMMENDED",
   "SpoilingType": "RECOMMENDED if the SpoilingState is `True`.",
   "SpoilingRFPhaseIncrement": 'RECOMMENDED if the SpoilingType is `"RF"` or `"COMBINED"`.',
   "SpoilingGradientMoment": 'RECOMMENDED if the SpoilingType is `"GRADIENT"` or `"COMBINED"`.',
   "SpoilingGradientDuration": 'RECOMMENDED if the SpoilingType is `"GRADIENT"` or `"COMBINED"`.',
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}PulseSequenceType (metadata),RECOMMENDED,string,"A general description of the pulse sequence used for the scan (for example, ""MPRAGE"", ""Gradient Echo EPI"", ""Spin Echo EPI"", ""Multiband gradient echo EPI"")."
{term}ScanningSequence (metadata),RECOMMENDED,string or array of strings,"Description of the type of data acquired. Corresponds to DICOM Tag 0018, 0020 Scanning Sequence."
{term}SequenceVariant (metadata),RECOMMENDED,string or array of strings,"Variant of the ScanningSequence. Corresponds to DICOM Tag 0018, 0021 Sequence Variant."
{term}ScanOptions (metadata),RECOMMENDED,string or array of strings,"Parameters of ScanningSequence. Corresponds to DICOM Tag 0018, 0022 Scan Options."
{term}SequenceName (metadata),RECOMMENDED,string,"Manufacturer's designation of the sequence name. Corresponds to DICOM Tag 0018, 0024 Sequence Name."
{term}PulseSequenceDetails (metadata),RECOMMENDED,string,"Information beyond pulse sequence type that identifies the specific pulse sequence used (for example, ""Standard Siemens Sequence distributed with the VB17 software"", ""Siemens WIP ### version #.##,"" or ""Sequence written by X using a version compiled on MM/DD/YYYY"")."
{term}NonlinearGradientCorrection (metadata),"RECOMMENDED, but REQUIRED if PET data are present",boolean,"Boolean stating if the image saved has been corrected for gradient nonlinearities by the scanner sequence. Must be one of: ""true"", ""false""."
{term}MRAcquisitionType (metadata),"RECOMMENDED, but REQUIRED for Arterial Spin Labeling",string,"Type of sequence readout. Corresponds to DICOM Tag 0018, 0023 MR Acquisition Type. Must be one of: ""2D"", ""3D""."
{term}MTState (metadata),RECOMMENDED,boolean,"Boolean stating whether the magnetization transfer pulse is applied. Corresponds to DICOM Tag 0018, 9020 Magnetization Transfer. Must be one of: ""true"", ""false""."
{term}MTOffsetFrequency (metadata),RECOMMENDED if the MTstate is True.,number,The frequency offset of the magnetization transfer pulse with respect to the central H1 Larmor frequency in Hertz (Hz).


(magnetic-resonance-imaging:common-metadata-fields:in-plane-spatial-encoding)=
### In-Plane Spatial Encoding

In [7]:
field_info = {
   "NumberShots": "RECOMMENDED",
   "ParallelReductionFactorInPlane": "RECOMMENDED",
   "ParallelAcquisitionTechnique": "RECOMMENDED",
   "PartialFourier": "RECOMMENDED",
   "PartialFourierDirection": "RECOMMENDED",
   "PhaseEncodingDirection": (
      "RECOMMENDED",
      "This parameter is REQUIRED if corresponding fieldmap data is present "
      "or when using multiple runs with different phase encoding directions "
      "(which can be later used for field inhomogeneity correction).",
   ),
   "EffectiveEchoSpacing": (
      "RECOMMENDED",
      "<sup>2</sup> This parameter is REQUIRED if corresponding fieldmap data is present.",
   ),
   "TotalReadoutTime": (
      "RECOMMENDED",
      "<sup>3</sup> This parameter is REQUIRED if corresponding 'field/distortion' maps "
      "acquired with opposing phase encoding directions are present "
      "(see [Case 4: Multiple phase encoded "
      "directions](#case-4-multiple-phase-encoded-directions-pepolar)).",
   ),
   "MixingTime": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}NumberShots (metadata),RECOMMENDED,number or array of numbers,"The number of RF excitations needed to reconstruct a slice or volume (may be referred to as partition). Please mind that this is not the same as Echo Train Length which denotes the number of k-space lines collected after excitation in a multi-echo readout. The data type array is applicable for specifying this parameter before and after the k-space center is sampled. Please see ""NumberShots"" metadata field in the qMRI appendix for corresponding calculations."
{term}ParallelReductionFactorInPlane (metadata),RECOMMENDED,number,"The parallel imaging (for instance, GRAPPA) factor. Use the denominator of the fraction of k-space encoded for each slice. For example, 2 means half of k-space is encoded. Corresponds to DICOM Tag 0018, 9069 Parallel Reduction Factor In-plane."
{term}ParallelAcquisitionTechnique (metadata),RECOMMENDED,string,"The type of parallel imaging used (for example ""GRAPPA"", ""SENSE""). Corresponds to DICOM Tag 0018, 9078 Parallel Acquisition Technique."
{term}PartialFourier (metadata),RECOMMENDED,number,"The fraction of partial Fourier information collected. Corresponds to DICOM Tag 0018, 9081 Partial Fourier."
{term}PartialFourierDirection (metadata),RECOMMENDED,string,"The direction where only partial Fourier information was collected. Corresponds to DICOM Tag 0018, 9036 Partial Fourier Direction."
{term}PhaseEncodingDirection (metadata),RECOMMENDED,string,"The letters i, j, k correspond to the first, second and third axis of the data in the NIFTI file. The polarity of the phase encoding is assumed to go from zero index to maximum index unless - sign is present (then the order is reversed - starting from the highest index instead of zero). PhaseEncodingDirection is defined as the direction along which phase is was modulated which may result in visible distortions. Note that this is not the same as the DICOM term InPlanePhaseEncodingDirection which can have ROW or COL values. This parameter is REQUIRED if corresponding fieldmap data is present or when using multiple runs with different phase encoding directions (which can be later used for field inhomogeneity correction).Must be one of: ""i"", ""j"", ""k"", ""i-"", ""j-"", ""k-""."
{term}EffectiveEchoSpacing (metadata),RECOMMENDED,number,"The ""effective"" sampling interval, specified in seconds, between lines in the phase-encoding direction, defined based on the size of the reconstructed image in the phase direction. It is frequently, but incorrectly, referred to as ""dwell time"" (see the ""DwellTime"" parameter for actual dwell time). It is required for unwarping distortions using field maps. Note that beyond just in-plane acceleration, a variety of other manipulations to the phase encoding need to be accounted for properly, including partial fourier, phase oversampling, phase resolution, phase field-of-view and interpolation. 2 This parameter is REQUIRED if corresponding fieldmap data is present.Must be a number greater than 0."
{term}TotalReadoutTime (metadata),RECOMMENDED,number,"This is actually the ""effective"" total readout time, defined as the readout duration, specified in seconds, that would have generated data with the given level of distortion. It is NOT the actual, physical duration of the readout train. If ""EffectiveEchoSpacing"" has been properly computed, it is just EffectiveEchoSpacing * (ReconMatrixPE - 1). 3 This parameter is REQUIRED if corresponding 'field/distortion' maps acquired with opposing phase encoding directions are present (see Case 4: Multiple phase encoded directions)."
{term}MixingTime (metadata),RECOMMENDED,number,"In the context of a stimulated- and spin-echo 3D EPI sequence for B1+ mapping, corresponds to the interval between spin- and stimulated-echo pulses. In the context of a diffusion-weighted double spin-echo sequence, corresponds to the interval between two successive diffusion sensitizing gradient

<sup>2</sup>Conveniently, for Siemens data, this value is easily obtained as
`1 / (BWPPPE * ReconMatrixPE)`, where BWPPPE is the
"BandwidthPerPixelPhaseEncode" in DICOM Tag 0019, 1028 and ReconMatrixPE is
the size of the actual reconstructed data in the phase direction (which is NOT
reflected in a single DICOM Tag for all possible aforementioned scan
manipulations). See [here](https://lcni.uoregon.edu/kb-articles/kb-0003) and
[here](https://github.com/neurolabusc/dcm_qa/tree/master/In/TotalReadoutTime)

<sup>3</sup>We use the time between the center of the first "effective" echo
and the center of the last "effective" echo, sometimes called the "FSL definition".

(magnetic-resonance-imaging:common-metadata-fields:timing-parameters)=
### Timing Parameters

In [8]:
field_info = {
   "EchoTime": "RECOMMENDED, but REQUIRED if corresponding fieldmap data is present, or the data comes from a multi echo sequence or Arterial Spin Labeling",
   "InversionTime": "RECOMMENDED",
   "SliceTiming": "RECOMMENDED, but REQUIRED for sparse sequences that do not have the `DelayTime` field set, and Arterial Spin Labeling with `MRAcquisitionType` set on `2D`.",
   "SliceEncodingDirection": "RECOMMENDED",
   "DwellTime": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}EchoTime (metadata),"RECOMMENDED, but REQUIRED if corresponding fieldmap data is present, or the data comes from a multi echo sequence or Arterial Spin Labeling",number or array of numbers,"The echo time (TE) for the acquisition, specified in seconds. Corresponds to DICOM Tag 0018, 0081 Echo Time (please note that the DICOM term is in milliseconds not seconds). The data type number may apply to files from any MRI modality concerned with a single value for this field, or to the files in a file collection where the value of this field is iterated using the echo entity. The data type array provides a value for each volume in a 4D dataset and should only be used when the volume timing is critical for interpretation of the data, such as in ASL or variable echo time fMRI sequences."
{term}InversionTime (metadata),RECOMMENDED,number,"The inversion time (TI) for the acquisition, specified in seconds. Inversion time is the time after the middle of inverting RF pulse to middle of excitation pulse to detect the amount of longitudinal magnetization. Corresponds to DICOM Tag 0018, 0082 Inversion Time (please note that the DICOM term is in milliseconds not seconds). Must be a number greater than 0."
{term}SliceTiming (metadata),"RECOMMENDED, but REQUIRED for sparse sequences that do not have the DelayTime field set, and Arterial Spin Labeling with MRAcquisitionType set on 2D.",array of numbers,"The time at which each slice was acquired within each volume (frame) of the acquisition. Slice timing is not slice order -- rather, it is a list of times containing the time (in seconds) of each slice acquisition in relation to the beginning of volume acquisition. The list goes through the slices along the slice axis in the slice encoding dimension (see below). Note that to ensure the proper interpretation of the ""SliceTiming"" field, it is important to check if the OPTIONAL SliceEncodingDirection exists. In particular, if ""SliceEncodingDirection"" is negative, the entries in ""SliceTiming"" are defined in reverse order with respect to the slice axis, such that the final entry in the ""SliceTiming"" list is the time of acquisition of slice 0. Without this parameter slice time correction will not be possible."
{term}SliceEncodingDirection (metadata),RECOMMENDED,string,"The axis of the NIfTI data along which slices were acquired, and the direction in which ""SliceTiming"" is defined with respect to. i, j, k identifiers correspond to the first, second and third axis of the data in the NIfTI file. A - sign indicates that the contents of ""SliceTiming"" are defined in reverse order - that is, the first entry corresponds to the slice with the largest index, and the final entry corresponds to slice index zero. When present, the axis defined by ""SliceEncodingDirection"" needs to be consistent with the slice_dim field in the NIfTI header. When absent, the entries in ""SliceTiming"" must be in the order of increasing slice index as defined by the NIfTI header. Must be one of: ""i"", ""j"", ""k"", ""i-"", ""j-"", ""k-""."
{term}DwellTime (metadata),RECOMMENDED,number,"Actual dwell time (in seconds) of the receiver per point in the readout direction, including any oversampling. For Siemens, this corresponds to DICOM field 0019, 1018 (in ns). This value is necessary for the optional readout distortion correction of anatomicals in the HCP Pipelines. It also usefully provides a handle on the readout bandwidth, which isn't captured in the other metadata tags. Not to be confused with ""EffectiveEchoSpacing"", and the frequent mislabeling of echo spacing (which is spacing in the phase encoding direction) as ""dwell time"" (which is spacing in the readout direction)."


(magnetic-resonance-imaging:common-metadata-fields:rf-&-contrast)=
### RF & Contrast

In [9]:
field_info = {
   "FlipAngle": "RECOMMENDED, but REQUIRED if `LookLocker` is set `true`",
   "NegativeContrast": "OPTIONAL",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}FlipAngle (metadata),"RECOMMENDED, but REQUIRED if LookLocker is set true",number or array of numbers,"Flip angle (FA) for the acquisition, specified in degrees. Corresponds to: DICOM Tag 0018, 1314 Flip Angle. The data type number may apply to files from any MRI modality concerned with a single value for this field, or to the files in a file collection where the value of this field is iterated using the flip entity. The data type array provides a value for each volume in a 4D dataset and should only be used when the volume timing is critical for interpretation of the data, such as in ASL or variable flip angle fMRI sequences."
{term}NegativeContrast (metadata),OPTIONAL,boolean,"true or false value specifying whether increasing voxel intensity (within sample voxels) denotes a decreased value with respect to the contrast suffix. This is commonly the case when Cerebral Blood Volume is estimated via usage of a contrast agent in conjunction with a T2* weighted acquisition protocol. Must be one of: ""true"", ""false""."


(magnetic-resonance-imaging:common-metadata-fields:slice-acceleration)=
### Slice Acceleration

In [10]:
field_info = {
   "MultibandAccelerationFactor": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}MultibandAccelerationFactor (metadata),RECOMMENDED,number,"The multiband factor, for multiband acquisitions."


(magnetic-resonance-imaging:common-metadata-fields:anatomical-landmarks)=
### Anatomical landmarks

Useful for multimodal co-registration with MEG, (S)EEG, TMS, and so on.

In [11]:
field_info = {
   "AnatomicalLandmarkCoordinates__mri": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}AnatomicalLandmarkCoordinates (metadata),RECOMMENDED,object of arrays,"Key:value pairs of any number of additional anatomical landmarks and their coordinates in voxel units (where first voxel has index 0,0,0) relative to the associated anatomical MRI (for example, {""AC"": [127,119,149], ""PC"": [128,93,141], ""IH"": [131,114,206]}, or {""NAS"": [127,213,139], ""LPA"": [52,113,96], ""RPA"": [202,113,91]}). Each array MUST contain three numeric values corresponding to x, y, and z axis of the coordinate system in that exact order."


(magnetic-resonance-imaging:common-metadata-fields:echo-planar-imaging-and-b0-mapping)=
### Echo-Planar Imaging and *B<sub>0</sub>* mapping

Echo-Planar Imaging (EPI) schemes typically used in the acquisition of
diffusion and functional MRI may also be *intended for* estimating the
*B<sub>0</sub>* field nonuniformity inside the scanner (in other words,
*mapping the field*) without the acquisition of additional MRI schemes
such as gradient-recalled echo (GRE) sequences that are stored under the
`fmap/` directory of the BIDS structure.

The modality labels `dwi` (under `dwi/`), `bold` (under `func/`),
`asl` (under `perf/`), `sbref` (under `dwi/`, `func/` or `perf/`), and
any modality under `fmap/` are allowed to encode the MR protocol intent for
fieldmap estimation using the following metadata:

In [12]:
field_info = {
   "B0FieldIdentifier": "RECOMMENDED",
   "B0FieldSource": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}B0FieldIdentifier (metadata),RECOMMENDED,string or array of strings,"The presence of this key states that this particular 3D or 4D image MAY be used for fieldmap estimation purposes. Each ""B0FieldIdentifier"" MUST be a unique string within one participant's tree, shared only by the images meant to be used as inputs for the estimation of a particular instance of the B0 field estimation. It is RECOMMENDED to derive this identifier from DICOM Tags, for example, DICOM tag 0018, 1030 Protocol Name, or DICOM tag 0018, 0024 Sequence Name when the former is not defined (for example, in GE devices.)"
{term}B0FieldSource (metadata),RECOMMENDED,string or array of strings,"At least one existing ""B0FieldIdentifier"" defined by images in the participant's tree. This field states the B0 field estimation designated by the ""B0FieldIdentifier"" that may be used to correct the dataset for distortions caused by B0 inhomogeneities. ""B0FieldSource"" and ""B0FieldIdentifier"" MAY both be present for images that are used to estimate their own B0 field, for example, in ""pepolar"" acquisitions."


(magnetic-resonance-imaging:common-metadata-fields:institution-information)=
### Institution information

In [13]:
field_info = {
   "InstitutionName": ("RECOMMENDED", "Corresponds to DICOM Tag 0008, 0080 `InstitutionName`."),
   "InstitutionAddress": ("RECOMMENDED", "Corresponds to DICOM Tag 0008, 0081 `InstitutionAddress`."),
   "InstitutionalDepartmentName": ("RECOMMENDED", "Corresponds to DICOM Tag 0008, 1040 `Institutional Department Name`.")
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}InstitutionName (metadata),RECOMMENDED,string,"The name of the institution in charge of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 0080 InstitutionName."
{term}InstitutionAddress (metadata),RECOMMENDED,string,"The address of the institution in charge of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 0081 InstitutionAddress."
{term}InstitutionalDepartmentName (metadata),RECOMMENDED,string,"The department in the institution in charge of the equipment that produced the measurements. Corresponds to DICOM Tag 0008, 1040 Institutional Department Name."


When adding additional metadata please use the CamelCase version of
[DICOM ontology terms](https://scicrunch.org/scicrunch/interlex/dashboard)
whenever possible. See also
[recommendations on JSON files](../02-common-principles.md#keyvalue-files-dictionaries).

(magnetic-resonance-imaging:anatomy-imaging-data)=
## Anatomy imaging data

In [14]:
codeblock = render.make_filename_template(schema_obj, datatypes=["anat"])
display(Markdown(codeblock))

Template:
```Text
sub-<label>/
    [ses-<label>/]
        anat/
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_part-<mag|phase|real|imag>]_<suffix>.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_part-<mag|phase|real|imag>]_<suffix>.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_<suffix>.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_<suffix>.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_mod-<label>]_defacemask.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_mod-<label>]_defacemask.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_echo-<index>[_part-<mag|phase|real|imag>]_MEGRE.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_echo-<index>[_part-<mag|phase|real|imag>]_MEGRE.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_echo-<index>[_part-<mag|phase|real|imag>]_MESE.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_echo-<index>[_part-<mag|phase|real|imag>]_MESE.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>]_flip-<index>[_part-<mag|phase|real|imag>]_VFA.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>]_flip-<index>[_part-<mag|phase|real|imag>]_VFA.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_inv-<index>[_part-<mag|phase|real|imag>]_IRT1.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_inv-<index>[_part-<mag|phase|real|imag>]_IRT1.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>][_flip-<index>]_inv-<index>[_part-<mag|phase|real|imag>]_MP2RAGE.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>][_flip-<index>]_inv-<index>[_part-<mag|phase|real|imag>]_MP2RAGE.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>]_flip-<index>_mt-<on|off>[_part-<mag|phase|real|imag>]_MPM.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>]_flip-<index>_mt-<on|off>[_part-<mag|phase|real|imag>]_MPM.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>]_flip-<index>_mt-<on|off>[_part-<mag|phase|real|imag>]_MTS.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>][_echo-<index>]_flip-<index>_mt-<on|off>[_part-<mag|phase|real|imag>]_MTS.nii[.gz]
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_mt-<on|off>[_part-<mag|phase|real|imag>]_MTR.json
            sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>][_rec-<label>][_run-<index>]_mt-<on|off>[_part-<mag|phase|real|imag>]_MTR.nii[.gz]
```

Currently supported non-parametric structural MR images include:

In [15]:
table = render.make_suffix_table(
   schema_obj,
   suffixes=[
      "T1w",
      "T2w",
      "PDw",
      "T2starw",
      "FLAIR",
      "inplaneT1",
      "inplaneT2",
      "PDT2",
      "UNIT1",
      "angio",
   ],
)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Name,suffix,Description
T1-weighted image,T1w,"In arbitrary units (arbitrary). The contrast of these images is mainly determined by spatial variations in the longitudinal relaxation time of the imaged specimen. In spin-echo sequences this contrast is achieved at relatively short repetition and echo times. To achieve this weighting in gradient-echo images, again, short repetition and echo times are selected; however, at relatively large flip angles. Another common approach to increase T1 weighting in gradient-echo images is to add an inversion preparation block to the beginning of the imaging sequence (for example, TurboFLASH or MP-RAGE)."
T2-weighted image,T2w,"In arbitrary units (arbitrary). The contrast of these images is mainly determined by spatial variations in the (true) transverse relaxation time of the imaged specimen. In spin-echo sequences this contrast is achieved at relatively long repetition and echo times. Generally, gradient echo sequences are not the most suitable option for achieving T2 weighting, as their contrast natively depends on T2-star rather than on T2."
Proton density (PD) weighted image,PDw,"In arbitrary units (arbitrary). The contrast of these images is mainly determined by spatial variations in the spin density (1H) of the imaged specimen. In spin-echo sequences this contrast is achieved at short repetition and long echo times. In a gradient-echo acquisition, PD weighting dominates the contrast at long repetition and short echo times, and at small flip angles."
T2star weighted image,T2starw,"In arbitrary units (arbitrary). The contrast of these images is mainly determined by spatial variations in the (observed) transverse relaxation time of the imaged specimen. In spin-echo sequences, this effect is negated as the excitation is followed by an inversion pulse. The contrast of gradient-echo images natively depends on T2-star effects. However, for T2-star variation to dominate the image contrast, gradient-echo acquisitions are carried out at long repetition and echo times, and at small flip angles."
Fluid attenuated inversion recovery image,FLAIR,"In arbitrary units (arbitrary). Structural images with predominant T2 contribution (also known as T2-FLAIR), in which signal from fluids (for example, CSF) is nulled out by adjusting inversion time, coupled with notably long repetition and echo times."
Inplane T1,inplaneT1,In arbitrary units (arbitrary). T1 weighted structural image matched to a functional (task) image.
Inplane T2,inplaneT2,In arbitrary units (arbitrary). T2 weighted structural image matched to a functional (task) image.
PD and T2 weighted image,PDT2,In arbitrary units (arbitrary). PDw and T2w images acquired using a dual echo FSE sequence through view sharing process (Johnson et al. 1994).
Homogeneous (flat) T1-weighted MP2RAGE image,UNIT1,"In arbitrary units (arbitrary). UNIT1 images are REQUIRED to use this suffix regardless of the method used to generate them. Note that although this image is T1-weighted, regions without MR signal will contain white salt-and-pepper noise that most segmentation algorithms will fail on. Therefore, it is important to dissociate it from T1w. Please see MP2RAGE specific notes in the qMRI appendix for further information."
Angiogram,angio,"Magnetic resonance angiography sequences focus on enhancing the contrast of blood vessels (generally arteries, but sometimes veins) against other tissue types."


If the structural images included in the dataset were defaced (to protect
identity of participants) one MAY provide the binary mask that was used to
remove facial features in the form of `_defacemask` files.
In such cases,  the OPTIONAL [`mod-<label>`](../99-appendices/09-entities.md#mod)
key/value pair corresponds to modality suffix,
such as T1w or inplaneT1, referenced by the defacemask image.
For example, `sub-01_mod-T1w_defacemask.nii.gz`.

If several scans with the same acquisition parameters are acquired in the same session,
they MUST be indexed with the [`run-<index>`](../99-appendices/09-entities.md#run) entity:
`_run-1`, `_run-2`, `_run-3`, and so on (only nonnegative integers are allowed as
run labels).

If different entities apply,
such as a different session indicated by [`ses-<label>`](../99-appendices/09-entities.md#ses),
or different acquisition parameters indicated by
[`acq-<label>`](../99-appendices/09-entities.md#acq),
then `run` is not needed to distinguish the scans and MAY be omitted.

The OPTIONAL [`acq-<label>`](../99-appendices/09-entities.md#acq)
key/value pair corresponds to a custom label the user
MAY use to distinguish a different set of parameters used for acquiring the same
modality. For example this should be used when a study includes two T1w images -
one full brain low resolution and one restricted field of view but high
resolution. In such case two files could have the following names:
`sub-01_acq-highres_T1w.nii.gz` and `sub-01_acq-lowres_T1w.nii.gz`, however the
user is free to choose any other label than `highres` and `lowres` as long as
they are consistent across subjects and sessions. In case different sequences
are used to record the same modality (for example, RARE and FLASH for T1w) this field
can also be used to make that distinction. At what level of detail to make the
distinction (for example, just between RARE and FLASH, or between RARE, FLASH, and
FLASHsubsampled) remains at the discretion of the researcher.

Similarly the OPTIONAL [`ce-<label>`](../99-appendices/09-entities.md#ce)
key/value can be used to distinguish
sequences using different contrast enhanced images. The label is the name of the
contrast agent. The key {term}`ContrastBolusIngredient` MAY be also be added in the
JSON file, with the same label.

Some meta information about the acquisition MAY be provided in an additional
JSON file. See {ref}`magnetic-resonance-imaging:common-metadata-fields` for a
list of terms and their definitions. There are also some OPTIONAL JSON
fields specific to anatomical scans:

In [16]:
field_info = {
   "ContrastBolusIngredient": "OPTIONAL",
   "RepetitionTimeExcitation": "OPTIONAL",
   "RepetitionTimePreparation": "OPTIONAL",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}ContrastBolusIngredient (metadata),OPTIONAL,string,"Active ingredient of agent. Corresponds to DICOM Tag 0018, 1048 Contrast/Bolus Ingredient. Must be one of: ""IODINE"", ""GADOLINIUM"", ""CARBON DIOXIDE"", ""BARIUM"", ""XENON""."
{term}RepetitionTimeExcitation (metadata),OPTIONAL,number,"The interval, in seconds, between two successive excitations. [DICOM Tag 0018, 0080](http://dicomlookup.com/lookup.asp?sw=Tnumber&q=(0018,0080) best refers to this parameter. This field may be used together with the ""RepetitionTimePreparation"" for certain use cases, such as MP2RAGE. Use RepetitionTimeExcitation (in combination with ""RepetitionTimePreparation"" if needed) for anatomy imaging data rather than ""RepetitionTime"" as it is already defined as the amount of time that it takes to acquire a single volume in the task imaging data section. Must be a number greater than or equal to 0."
{term}RepetitionTimePreparation (metadata),OPTIONAL,number or array of numbers,"The interval, in seconds, that it takes a preparation pulse block to re-appear at the beginning of the succeeding (essentially identical) pulse sequence block. The data type number may apply to files from any MRI modality concerned with a single value for this field. The data type array provides a value for each volume in a 4D dataset and should only be used when the volume timing is critical for interpretation of the data, such as in ASL."


The [`part-<label>`](../99-appendices/09-entities.md#part) key/value pair is
used to indicate which component of the complex representation of the MRI
signal is represented in voxel data.
This entity is associated with the DICOM Tag `0008, 9208`.
Allowed label values for this entity are `phase`, `mag`, `real` and `imag`,
which are typically used in `part-mag`/`part-phase` or `part-real`/`part-imag`
pairs of files.
For example:

{{ MACROS___make_filetree_example(
   {
   "sub-01": {
      "anat": {
         "sub-01_part-mag_T1w.nii.gz": "",
         "sub-01_part-mag_T1w.json": "",
         "sub-01_part-phase_T1w.nii.gz": "",
         "sub-01_part-phase_T1w.json": "",
         },
      }
   }
) }}

Phase images MAY be in radians or in arbitrary units.
The sidecar JSON file MUST include the units of the `phase` image.
The possible options are `rad` or `arbitrary`.

For example, for `sub-01_part-phase_T1w.json`:

```Text
{
   "Units": "rad"
}
```

When there is only a magnitude image of a given type, the `part` key MAY be omitted.

Similarly, the OPTIONAL [`rec-<label>`](../99-appendices/09-entities.md#rec)
key/value can be used to distinguish
different reconstruction algorithms (for example ones using motion correction).

Structural MR images whose intensity is represented in a non-arbitrary scale
constitute parametric maps. Currently supported parametric maps include:

{{ MACROS___make_suffix_table(
      [
         "T1map",
         "R1map",
         "T2map",
         "R2map",
         "T2starmap",
         "R2starmap",
         "PDmap",
         "MTRmap",
         "MTsat",
         "T1rho",
         "MWFmap",
         "MTVmap",
         "PDT2map",
         "Chimap",
         "TB1map",
         "RB1map",
         "S0map",
         "M0map",
      ]
   )
}}

Parametric images listed in the table above are typically generated by
processing a [file collection](../02-common-principles.md#entity-linked-file-collections).
Please visit the [file collections appendix](../99-appendices/10-file-collections.md) to see the
list of suffixes available for quantitative MRI (qMRI) applications associated
with these maps.
For any other details on the organization of parametric maps, their
recommended metadata fields, and the application specific entity or
metadata requirement levels of [file collections](../99-appendices/10-file-collections.md) that can generate
them, visit the [qMRI appendix](../99-appendices/11-qmri.md).

(magnetic-resonance-imaging:anatomy-imaging-data:deprecated-suffixes)=
### Deprecated suffixes

Some suffixes that were available in versions of the specification prior to
1.5.0 have been deprecated.
These suffixes are ambiguous and have been superseded by more precise conventions.
Therefore, they are not recommended for use in new datasets.
They are, however, still valid suffixes, to maintain backwards compatibility.

The following suffixes are valid, but SHOULD NOT be used for new BIDS compatible
datasets (created after version 1.5.0.):

{{ MACROS___make_suffix_table(
      [
         "T2star",
         "FLASH",
         "PD",
      ]
   )
}}

(magnetic-resonance-imaging:task-imaging-data)=
## Task (including resting state) imaging data

Currently supported image contrasts include:

{{ MACROS___make_suffix_table(
      [
         "bold",
         "cbv",
         "phase",
      ]
   )
}}

{{ MACROS___make_filename_template(datatypes=["func"]) }}

Functional imaging consists of techniques that support rapid temporal repetition.
This includes but is not limited to task based fMRI
as well as resting state fMRI, which is treated like any other task. For task
based fMRI a corresponding task events file (see below) MUST be provided
(please note that this file is not necessary for resting state scans). For
multiband acquisitions, one MAY also save the single-band reference image as
type `sbref` (for example, `sub-control01_task-nback_sbref.nii.gz`).

Each task has a unique label that MUST only consist of letters and/or numbers
(other characters, including spaces and underscores, are not allowed) with the
[`task-<label>`](../99-appendices/09-entities.md#task) key/value pair.
Those labels MUST be consistent across subjects and sessions.

If more than one run of the same task has been acquired the
[`run-<index>`](../99-appendices/09-entities.md#run) key/value pair MUST be used:
`_run-1`, `_run-2`, `_run-3`, and so on. If only one run was acquired the
`run-<index>` can be omitted. In the context of functional imaging a run is
defined as the same task, but in some cases it can mean different set of stimuli
(for example randomized order) and participant responses.

The OPTIONAL [`acq-<label>`](../99-appendices/09-entities.md#acq)
key/value pair corresponds to a custom label one may
use to distinguish different set of parameters used for acquiring the same task.
For example this should be used when a study includes two resting state images -
one single band and one multiband. In such case two files could have the
following names: `sub-01_task-rest_acq-singleband_bold.nii.gz` and
`sub-01_task-rest_acq-multiband_bold.nii.gz`, however the user is MAY choose any
other label than `singleband` and `multiband` as long as they are consistent
across subjects and sessions and consist only of the legal label characters.

Similarly the OPTIONAL [`ce-<label>`](../99-appendices/09-entities.md#ce)
key/value can be used to distinguish
sequences using different contrast enhanced images. The label is the name of the
contrast agent. The key ContrastBolusIngredient MAY be also be added in the JSON
file, with the same label.

Similarly the OPTIONAL [`rec-<label>`](../99-appendices/09-entities.md#rec)
key/value can be used to distinguish
different reconstruction algorithms (for example ones using motion correction).

Similarly the OPTIONAL [`dir-<label>`](../99-appendices/09-entities.md#dir)
and [`rec-<label>`](../99-appendices/09-entities.md#rec) key/values
can be used to distinguish different phase-encoding directions and
reconstruction algorithms (for example ones using motion correction).
See [`fmap` Case 4](01-magnetic-resonance-imaging-data.md#case-4-multiple-phase-encoded-directions-pepolar)
for more information on `dir` field specification.

Multi-echo data MUST be split into one file per echo using the
[`echo-<index>`](../99-appendices/09-entities.md#echo) key-value pair. For example:

{{ MACROS___make_filetree_example(
   {
   "sub-01": {
      "func": {
         "sub-01_task-cuedSGT_run-1_echo-1_bold.nii.gz": "",
         "sub-01_task-cuedSGT_run-1_echo-1_bold.json": "",
         "sub-01_task-cuedSGT_run-1_echo-2_bold.nii.gz": "",
         "sub-01_task-cuedSGT_run-1_echo-2_bold.json": "",
         "sub-01_task-cuedSGT_run-1_echo-3_bold.nii.gz": "",
         "sub-01_task-cuedSGT_run-1_echo-3_bold.json": "",
         },
      }
   }
) }}

Please note that the `<index>` denotes the number/index (in the form of a
nonnegative integer) of the echo not the echo time value which needs to be stored in the
field EchoTime of the separate JSON file.

Complex-valued data MUST be split into one file for each data type.
For BOLD data, there are separate suffixes for magnitude (`_bold`) and phase
(`_phase`) data, but the `_phase` suffix is [deprecated](../02-common-principles.md#definitions).
Newly generated datasets SHOULD NOT use the `_phase` suffix, and the suffix will be removed
from the specification in the next major release.
For backwards compatibility, `_phase` is considered equivalent to `_part-phase_bold`.
When the `_phase` suffix is not used, each file shares the same
name with the exception of the `part-<mag|phase>` or `part-<real|imag>` key/value.

For example:

{{ MACROS___make_filetree_example(
   {
   "sub-01": {
      "func": {
         "sub-01_task-cuedSGT_part-mag_bold.nii.gz": "",
         "sub-01_task-cuedSGT_part-mag_bold.json": "",
         "sub-01_task-cuedSGT_part-phase_bold.nii.gz": "",
         "sub-01_task-cuedSGT_part-phase_bold.json": "",
         "sub-01_task-cuedSGT_part-mag_sbref.nii.gz": "",
         "sub-01_task-cuedSGT_part-mag_sbref.json": "",
         "sub-01_task-cuedSGT_part-phase_sbref.nii.gz": "",
         "sub-01_task-cuedSGT_part-phase_sbref.json": "",
         },
      },
   }
) }}

Some meta information about the acquisition MUST be provided in an additional
JSON file.

(magnetic-resonance-imaging:task-imaging-data:required-fields)=
### Required fields

In [17]:
field_info = {
   "RepetitionTime": "REQUIRED",
   "VolumeTiming": "REQUIRED",
   "TaskName": ("REQUIRED", "A RECOMMENDED convention is to name resting state task using labels beginning with `rest`.")
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}RepetitionTime (metadata),REQUIRED,number,"The time in seconds between the beginning of an acquisition of one volume and the beginning of acquisition of the volume following it (TR). When used in the context of functional acquisitions this parameter best corresponds to DICOM Tag 0020, 0110: the ""time delta between images in a dynamic of functional set of images"" but may also be found in DICOM Tag 0018, 0080: ""the period of time in msec between the beginning of a pulse sequence and the beginning of the succeeding (essentially identical) pulse sequence"". This definition includes time between scans (when no data has been acquired) in case of sparse acquisition schemes. This value MUST be consistent with the 'pixdim[4]' field (after accounting for units stored in 'xyzt_units' field) in the NIfTI header. This field is mutually exclusive with VolumeTiming. Must be a number greater than 0."
{term}VolumeTiming (metadata),REQUIRED,array of numbers,"The time at which each volume was acquired during the acquisition. It is described using a list of times referring to the onset of each volume in the BOLD series. The list must have the same length as the BOLD series, and the values must be non-negative and monotonically increasing. This field is mutually exclusive with ""RepetitionTime"" and ""DelayTime"". If defined, this requires acquisition time (TA) be defined via either ""SliceTiming"" or ""AcquisitionDuration"" be defined."
{term}TaskName (metadata),REQUIRED,string,"Name of the task. No two tasks should have the same name. The task label included in the file name is derived from this ""TaskName"" field by removing all non-alphanumeric ([a-zA-Z0-9]) characters. For example ""TaskName"" ""faces n-back"" will correspond to task label facesnback. A RECOMMENDED convention is to name resting state task using labels beginning with rest."


For the fields described above and in the following section, the term "Volume"
refers to a reconstruction of the object being imaged (for example, brain or part of a
brain). In case of multiple channels in a coil, the term "Volume" refers to a
combined image rather than an image from each coil.

(magnetic-resonance-imaging:task-imaging-data:other-recommended-metadata)=
### Other RECOMMENDED metadata

(magnetic-resonance-imaging:task-imaging-data:other-recommended-metadata:timing-parameters)=
#### Timing Parameters

In [18]:
field_info = {
   "NumberOfVolumesDiscardedByScanner": "RECOMMENDED",
   "NumberOfVolumesDiscardedByUser": "RECOMMENDED",
   "DelayTime": "RECOMMENDED",
   "AcquisitionDuration": 'RECOMMENDED, but REQUIRED for sequences that are described with the `VolumeTiming` field and that do not have the `SliceTiming` field set to allow for accurate calculation of "acquisition time"',
   "DelayAfterTrigger": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}NumberOfVolumesDiscardedByScanner (metadata),RECOMMENDED,integer,"Number of volumes (""dummy scans"") discarded by the scanner (as opposed to those discarded by the user post hoc) before saving the imaging file. For example, a sequence that automatically discards the first 4 volumes before saving would have this field as 4. A sequence that does not discard dummy scans would have this set to 0. Please note that the onsets recorded in the events.tsv file should always refer to the beginning of the acquisition of the first volume in the corresponding imaging file - independent of the value of ""NumberOfVolumesDiscardedByScanner"" field. Must be a number greater than or equal to 0."
{term}NumberOfVolumesDiscardedByUser (metadata),RECOMMENDED,integer,"Number of volumes (""dummy scans"") discarded by the user before including the file in the dataset. If possible, including all of the volumes is strongly recommended. Please note that the onsets recorded in the events.tsv file should always refer to the beginning of the acquisition of the first volume in the corresponding imaging file - independent of the value of ""NumberOfVolumesDiscardedByUser"" field. Must be a number greater than or equal to 0."
{term}DelayTime (metadata),RECOMMENDED,number,"User specified time (in seconds) to delay the acquisition of data for the following volume. If the field is not present it is assumed to be set to zero. Corresponds to Siemens CSA header field lDelayTimeInTR. This field is REQUIRED for sparse sequences using the ""RepetitionTime"" field that do not have the ""SliceTiming"" field set to allowed for accurate calculation of ""acquisition time"". This field is mutually exclusive with ""VolumeTiming""."
{term}AcquisitionDuration (metadata),"RECOMMENDED, but REQUIRED for sequences that are described with the VolumeTiming field and that do not have the SliceTiming field set to allow for accurate calculation of ""acquisition time""",number,"Duration (in seconds) of volume acquisition. Corresponds to DICOM Tag 0018, 9073 Acquisition Duration. This field is mutually exclusive with ""RepetitionTime"". Must be a number greater than 0."
{term}DelayAfterTrigger (metadata),RECOMMENDED,number,"Duration (in seconds) from trigger delivery to scan onset. This delay is commonly caused by adjustments and loading times. This specification is entirely independent of ""NumberOfVolumesDiscardedByScanner"" or ""NumberOfVolumesDiscardedByUser"", as the delay precedes the acquisition."


The following table recapitulates the different ways that specific fields have
to be populated for functional sequences. Note that all these options can be
used for non sparse sequences but that only options B, D and E are valid for
sparse sequences.

|          | **`RepetitionTime`** | **`SliceTiming`** | **`AcquisitionDuration`** | **`DelayTime`** | **`VolumeTiming`** |
| -------- | -------------------- | ----------------- | ------------------------- | --------------- | ------------------ |
| option A |     \[ X ]           |                   |         \[ ]              |                 |     \[ ]           |
| option B |      \[ ]            |    \[ X ]         |                           |    \[ ]         |    \[ X ]          |
| option C |      \[ ]            |                   |        \[ X ]             |    \[ ]         |    \[ X ]          |
| option D |     \[ X ]           |    \[ X ]         |         \[ ]              |                 |     \[ ]           |
| option E |     \[ X ]           |                   |         \[ ]              |   \[ X ]        |     \[ ]           |

**Legend**

-   \[ X ] --> MUST be defined
-   \[  \] --> MUST NOT be defined
-   empty cell --> MAY be specified

(magnetic-resonance-imaging:task-imaging-data:other-recommended-metadata:fmri-task-information)=
#### fMRI task information

In [19]:
field_info = {
   "Instructions": ("RECOMMENDED", "This is especially important in context of resting state recordings and distinguishing between eyes open and eyes closed paradigms."),
   "TaskDescription": "RECOMMENDED",
   "CogAtlasID": "RECOMMENDED",
   "CogPOID": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}Instructions (metadata),RECOMMENDED,string,Text of the instructions given to participants before the recording. This is especially important in context of resting state recordings and distinguishing between eyes open and eyes closed paradigms.
{term}TaskDescription (metadata),RECOMMENDED,string,Longer description of the task.
{term}CogAtlasID (metadata),RECOMMENDED,string,URI of the corresponding Cognitive Atlas Task term.
{term}CogPOID (metadata),RECOMMENDED,string,URI of the corresponding CogPO term.


See [Common metadata fields](#common-metadata-fields) for a list of
additional terms and their definitions.

Example:

{{ MACROS___make_filetree_example(
   {
   "sub-01": {
      "func": {
         "sub-control01_task-nback_bold.json": "",
         },
      }
   }
) }}

```JSON
{
   "TaskName": "N Back",
   "RepetitionTime": 0.8,
   "EchoTime": 0.03,
   "FlipAngle": 78,
   "SliceTiming": [0.0, 0.2, 0.4, 0.6, 0.0, 0.2, 0.4, 0.6, 0.0, 0.2, 0.4, 0.6, 0.0, 0.2, 0.4, 0.6],
   "MultibandAccelerationFactor": 4,
   "ParallelReductionFactorInPlane": 2,
   "PhaseEncodingDirection": "j",
   "InstitutionName": "Stanford University",
   "InstitutionAddress": "450 Serra Mall, Stanford, CA 94305-2004, USA",
   "DeviceSerialNumber": "11035",
   "B0FieldSource": ["phasediff_fmap0", "pepolar_fmap0"]
}
```

If this information is the same for all participants, sessions and runs it can
be provided in `task-<label>_bold.json` (in the root directory of the
dataset). However, if the information differs between subjects/runs it can be
specified in the
`sub-<label>/func/sub-<label>_task-<label>[_acq-<label>][_run-<index>]_bold.json` file.
If both files are specified fields from the file corresponding to a particular
participant, task and run takes precedence.

(magnetic-resonance-imaging:diffusion-imaging-data)=
## Diffusion imaging data

Several [example datasets](https://github.com/bids-standard/bids-examples)
contain diffusion imaging data formatted using this specification
and that can be used for practical guidance when curating a new dataset:

-   [`genetics_ukbb`](https://github.com/bids-standard/bids-examples/tree/master/genetics_ukbb)
-   [`eeg_rest_fmri`](https://github.com/bids-standard/bids-examples/tree/master/eeg_rest_fmri)
-   [`ds114`](https://github.com/bids-standard/bids-examples/tree/master/ds114)
-   [`ds000117`](https://github.com/bids-standard/bids-examples/tree/master/ds000117)

Diffusion-weighted imaging data acquired for a participant.
Currently supported image types include:

{{ MACROS___make_suffix_table(
      [
         "dwi",
         "sbref",
      ]
   )
}}

{{ MACROS___make_filename_template(datatypes=["dwi"]) }}

If more than one run of the same acquisition and direction has been acquired, the
[`run-<index>`](../99-appendices/09-entities.md#run) key/value pair MUST be used:
`_run-1`, `_run-2`, `_run-3` (and so forth.)
When there is only one scan of a given acquisition and direction, the run key MAY be
omitted.
The [`run-<index>`](../99-appendices/09-entities.md#run) key/value pair is RECOMMENDED
to encode the splits of multipart DWI scans (see [below](#multipart-split-dwi-schemes).)

The OPTIONAL [`acq-<label>`](../99-appendices/09-entities.md#acq)
key/value pair corresponds to a custom label the user may use to
distinguish different sets of parameters.

The OPTIONAL [`dir-<label>`](../99-appendices/09-entities.md#dir)
key/value pair corresponds to a custom label the user may use to
distinguish different sets of phase-encoding directions.

**Combining multi- and single-band acquisitions**.
The single-band reference image MAY be stored with suffix `sbref` (for example,
`dwi/sub-control01_sbref.nii[.gz]`) as long as the image has no corresponding
[gradient information (`[*_]dwi.bval` and `[*_]dwi.bvec` sidecar files)](#required-gradient-orientation-information)
to be stored.

Otherwise, if some gradient information is associated to the single-band diffusion
image and a multi-band diffusion image also exists, the `acq-<label>` key/value pair
MUST be used to distinguish both images.
In such a case, two files could have the following names:
`sub-01_acq-singleband_dwi.nii.gz` and `sub-01_acq-multiband_dwi.nii.gz`.
The user is free to choose any other label than `singleband` and
`multiband`, as long as they are consistent across subjects and sessions.

(magnetic-resonance-imaging:diffusion-imaging-data:required-gradient-orientation-information)=
### REQUIRED gradient orientation information

The REQUIRED gradient orientation information corresponding to a DWI acquisition
MUST be stored using `[*_]dwi.bval` and `[*_]dwi.bvec` pairs of files.
The `[*_]dwi.bval` and `[*_]dwi.bvec` files MAY be saved on any level of the directory structure
and thus define those values for all sessions and/or subjects in one place (see
[the inheritance principle](../02-common-principles.md#the-inheritance-principle)).

As an exception to the [common principles](../02-common-principles.md#definitions)
that parameters are constant across runs, the gradient table information (stored
within the `[*_]dwi.bval` and `[*_]dwi.bvec` files) MAY change across DWI runs.

**Gradient orientation file formats**.
The `[*_]dwi.bval` and `[*_]dwi.bvec` files MUST follow the
[FSL format](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FDT/UserGuide#DTIFIT):
The `[*_]dwi.bvec` file contains 3 rows with *N* space-delimited floating-point numbers
(corresponding to the *N* volumes in the corresponding NIfTI file.)
The first row contains the *x* elements, the second row contains the *y* elements and
the third row contains the *z* elements of a unit vector in the direction of the applied
diffusion gradient, where the *i*-th elements in each row correspond together to
the *i*-th volume, with `[0,0,0]` for *non-diffusion-weighted* (also called *b*=0 or *low-b*)
volumes.
Following the FSL format for the `[*_]dwi.bvec` specification, the coordinate system of
the *b* vectors MUST be defined with respect to the coordinate system defined by
the header of the corresponding `_dwi` NIfTI file and not the scanner's device
coordinate system (see [Coordinate systems](../99-appendices/08-coordinate-systems.md)).
The most relevant limitation imposed by this choice is that the gradient information cannot
be directly stored in this format if the scanner generates *b*-vectors in *scanner coordinates*.

Example of `[*_]dwi.bvec` file, with *N*=6, with two *b*=0 volumes in the beginning:

```Text
0 0 0.021828 -0.015425 -0.70918 -0.2465
0 0 0.80242 0.22098 -0.00063106 0.1043
0 0 -0.59636 0.97516 -0.70503 -0.96351
```

The `[*_]dwi.bval` file contains the *b*-values (in s/mm<sup>2</sup>) corresponding to the
volumes in the relevant NIfTI file), with 0 designating *b*=0 volumes, space-delimited.

Example of `[*_]dwi.bval` file, corresponding to the previous `[*_]dwi.bvec` example:

```Text
0 0 2000 2000 1000 1000
```

(magnetic-resonance-imaging:diffusion-imaging-data:multipart-dwi-schemes)=
### Multipart (split) DWI schemes

Some MR schemes cannot be acquired directly by some scanner devices,
requiring to generate several DWI runs that were originally meant to belong
in a single one.
For instance, some GE scanners cannot collect more than &asymp;160 volumes
in a single run under fast-changing gradients, so acquiring *HCP-style*
diffusion images will require splitting the DWI scheme in several runs.
Because researchers will generally optimize the data splits, these will likely
not be able to be directly concatenated.
BIDS permits defining arbitrary groupings of these multipart scans with the
following metadata:

In [20]:
field_info = {
   "MultipartID": "REQUIRED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}MultipartID (metadata),REQUIRED,string,A unique (per participant) label tagging DWI runs that are part of a multipart scan.


JSON example:

```JSON
{
  "MultipartID": "dwi_1"
}
```

For instance, if there are two phase-encoding directions (`AP`, `PA`), and
two runs each, and the intent of the researcher is that all of them are
part of a unique multipart scan, then they will tag all four runs with the
same `MultipartID` (shown at the right-hand side of the file listing):

{{ MACROS___make_filetree_example(
   {
   "sub-1": {
      "dwi                               # MultipartID": {
         "sub-1_dir-AP_run-1_dwi.nii.gz": " # dwi_1",
         "sub-1_dir-AP_run-2_dwi.nii.gz": " # dwi_1",
         "sub-1_dir-PA_run-1_dwi.nii.gz": " # dwi_1",
         "sub-1_dir-PA_run-2_dwi.nii.gz": " # dwi_1",
         }
      }
   }
) }}

If, conversely, the researcher wanted to store two multipart scans, one possibility
is to combine matching phase-encoding directions:

{{ MACROS___make_filetree_example(
   {
   "sub-1": {
      "dwi                               # MultipartID":{
            "sub-1_dir-AP_run-1_dwi.nii.gz": " # dwi_1",
            "sub-1_dir-AP_run-2_dwi.nii.gz": " # dwi_1",
            "sub-1_dir-PA_run-1_dwi.nii.gz": " # dwi_2",
            "sub-1_dir-PA_run-2_dwi.nii.gz": " # dwi_2",
      },
      }
   }
) }}

Alternatively, the researcher's intent could be combining opposed phase-encoding
runs instead:

{{ MACROS___make_filetree_example(
   {
   "sub-1": {
      "dwi                               # MultipartID":{
            "sub-1_dir-AP_run-1_dwi.nii.gz": " # dwi_1",
            "sub-1_dir-AP_run-2_dwi.nii.gz": " # dwi_2",
            "sub-1_dir-PA_run-1_dwi.nii.gz": " # dwi_1",
            "sub-1_dir-PA_run-2_dwi.nii.gz": " # dwi_2",
      },
      }
   }
) }}

The `MultipartID` metadata MAY be used with the
[`acq-<label>`](../99-appendices/09-entities.md#acq) key/value pair, for example:

{{ MACROS___make_filetree_example(
   {
   "sub-1": {
      "dwi                                   # MultipartID":{
         "sub-1_acq-shell1_run-1_dwi.nii.gz": " # dwi_1",
         "sub-1_acq-shell1_run-2_dwi.nii.gz": " # dwi_2",
         "sub-1_acq-shell2_run-1_dwi.nii.gz": " # dwi_1",
         "sub-1_acq-shell2_run-2_dwi.nii.gz": " # dwi_2",
         },
      }
   }
) }}

(magnetic-resonance-imaging:diffusion-imaging-data:other-recommended-metadata)=
### Other RECOMMENDED metadata

The `PhaseEncodingDirection` and `TotalReadoutTime` metadata
fields are RECOMMENDED to enable the correction of geometrical distortions
with [fieldmap information](#fieldmap-data).
See [Common metadata fields](#common-metadata-fields) for a list of
additional terms that can be included in the corresponding JSON file.

JSON example:

```JSON
{
  "PhaseEncodingDirection": "j-",
  "TotalReadoutTime": 0.095,
  "B0FieldSource": ["phasediff_fmap0", "pepolar_fmap0"]
}
```

(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data)=
## Arterial Spin Labeling perfusion data

Several [example ASL datasets](https://github.com/bids-standard/bids-examples#asl-datasets)
have been formatted using this specification
and can be used for practical guidance when curating a new dataset.

{{ MACROS___make_filename_template(datatypes=["perf"]) }}

The complete ASL time series should be stored as a 4D NIfTI file in the original acquisition order,
accompanied by two ancillary files: `*_asl.json` and `*_aslcontext.tsv`.

(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:aslcontext.tsv)=
### `*_aslcontext.tsv`

The `*_aslcontext.tsv` table consists of a single column of labels identifying the
`volume_type` of each volume in the corresponding `*_asl.nii[.gz]` file.
Volume types are defined in the following table, based on DICOM Tag 0018, 9257 `ASL Context`.
Note that the volume_types `control` and  `label` within BIDS only serve
to specify the magnetization state of the blood and thus the ASL subtraction order.
See [Appendix XII - ASL](../99-appendices/12-arterial-spin-labeling.md#which-image-is-control-and-which-is-label) for more information on `control` and  `label`.

| **volume_type** | **Definition**                                                                                                                                                                         |
| --------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| control         | The control image is acquired in the exact same way as the label image, except that the magnetization of the blood flowing into the imaging region has not been inverted.              |
| label           | The label image is acquired in the exact same way as the control image, except that the blood magnetization flowing into the imaging region has been inverted.                         |
| m0scan          | The M0 image is a calibration image, used to estimate the equilibrium magnetization of blood.                                                                                          |
| deltam          | The deltaM image is a perfusion-weighted image, obtained by the subtraction of `control` - `label`.                                                                                    |
| cbf             | The cerebral blood flow (CBF) image is produced by dividing the deltaM by the M0, quantified into `mL/100g/min` (See also [doi:10.1002/mrm.25197](https://doi.org/10.1002/mrm.25197)). |

If the `control` and `label` images are not available,
their derivative `deltam` should be stored within the `*_asl.nii[.gz]`
and specified in the `*_aslcontext.tsv` instead.
If the `deltam` is not available,
`cbf` should be stored within the `*_asl.nii[.gz]` and specified in the `*_aslcontext.tsv`.
When `cbf` is stored within the `*_asl.nii[.gz]`,
its units need to be specified in the `*_asl.json` as well.
Note that the raw images, including the `m0scan`, may also be used for quality control.
See [Appendix XII - ASL](../99-appendices/12-arterial-spin-labeling.md#_aslcontexttsv-three-possible-cases) for examples of the three possible cases, in order of decreasing preference.

(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:scaling)=
### Scaling

The `*_asl.nii.gz` and `*_m0scan.nii.gz` should contain appropriately scaled data, and no additional scaling factors are allowed other than the scale slope in the respective
NIfTI headers.

(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:m0)=
### M0

The `m0scan` can either be stored inside the 4D ASL time-series NIfTI file
or as a separate NIfTI file,
depending on whether it was acquired within the ASL time-series or as a separate scan.
These and other M0 options are specified in the REQUIRED `M0Type` field of the `*_asl.json` file.
It can also be stored under `fmap/sub-<label>[_ses-<label>][_acq-<label>][_ce-<label>]_dir-<label>[_run-<index>]_m0scan.nii[.gz]`,
when the [pepolar approach](#case-4-multiple-phase-encoded-directions-pepolar) is used.

(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:asl.json-file)=
### `*_asl.json` file

Depending on the method used for ASL acquisition ((P)CASL or PASL)
different metadata fields are applicable.
Additionally, some common metadata fields are REQUIRED for the `*_asl.json`:
`MagneticFieldStrength`, `MRAcquisitionType`, `EchoTime`,
`SliceTiming` in case `MRAcquisitionType` is defined as 2D,
`RepetitionTimePreparation`, and `FlipAngle` in case `LookLocker` is `true`.
See [Appendix XII - ASL](../99-appendices/12-arterial-spin-labeling.md#summary-image-of-the-most-common-asl-sequences) for more information on the most common ASL sequences.

(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:asl.json-file:common-metadata-fields-applicable-to-both-pcasl-and-pasl)=
#### Common metadata fields applicable to both (P)CASL and PASL

In [21]:
field_info = {
   "ArterialSpinLabelingType": "REQUIRED",
   "PostLabelingDelay": "REQUIRED",
   "BackgroundSuppression": "REQUIRED",
   "M0Type": "REQUIRED",
   "TotalAcquiredPairs": "REQUIRED",
   "VascularCrushing": "RECOMMENDED",
   "AcquisitionVoxelSize": "RECOMMENDED",
   "M0Estimate": "OPTIONAL, but REQUIRED when `M0Type` is defined as `Estimate`",
   "BackgroundSuppressionNumberPulses": "OPTIONAL, RECOMMENDED if `BackgroundSuppression` is `true`",
   "BackgroundSuppressionPulseTime": "OPTIONAL, RECOMMENDED if `BackgroundSuppression` is `true`",
   "VascularCrushingVENC": "OPTIONAL, RECOMMENDED if `VascularCrushing` is `true`",
   "LabelingOrientation": "RECOMMENDED",
   "LabelingDistance": "RECOMMENDED",
   "LabelingLocationDescription": "RECOMMENDED",
   "LookLocker": "OPTIONAL",
   "LabelingEfficiency": "OPTIONAL",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}ArterialSpinLabelingType (metadata),REQUIRED,string,"The arterial spin labeling type. Must be one of: ""CASL"", ""PCASL"", ""PASL""."
{term}PostLabelingDelay (metadata),REQUIRED,number or array of numbers,"This is the postlabeling delay (PLD) time, in seconds, after the end of the labeling (for ""CASL"" or ""PCASL"") or middle of the labeling pulse (for ""PASL"") until the middle of the excitation pulse applied to the imaging slab (for 3D acquisition) or first slice (for 2D acquisition). Can be a number (for a single-PLD time series) or an array of numbers (for multi-PLD and Look-Locker). In the latter case, the array of numbers contains the PLD of each volume, namely each control and label, in the acquisition order. Any image within the time-series without a PLD, for example an m0scan, is indicated by a zero. Based on DICOM Tags 0018, 9079 Inversion Times and 0018, 0082 InversionTime."
{term}BackgroundSuppression (metadata),REQUIRED,boolean,"Boolean indicating if background suppression is used. Must be one of: ""true"", ""false""."
{term}M0Type (metadata),REQUIRED,string,"Describes the presence of M0 information. ""Separate"" means that a separate *_m0scan.nii[.gz] is present. ""Included"" means that an m0scan volume is contained within the current *_asl.nii[.gz]. ""Estimate"" means that a single whole-brain M0 value is provided. ""Absent"" means that no specific M0 information is present. Must be one of: ""Separate"", ""Included"", ""Estimate"", ""Absent""."
{term}TotalAcquiredPairs (metadata),REQUIRED,number,The total number of acquired control-label pairs. A single pair consists of a single control and a single label image. Must be a number greater than 0.
{term}VascularCrushing (metadata),RECOMMENDED,boolean,"Boolean indicating if Vascular Crushing is used. Corresponds to DICOM Tag 0018, 9259 ASL Crusher Flag. Must be one of: ""true"", ""false""."
{term}AcquisitionVoxelSize (metadata),RECOMMENDED,array of numbers,"An array of numbers with a length of 3, in millimeters. This parameter denotes the original acquisition voxel size, excluding any inter-slice gaps and before any interpolation or resampling within reconstruction or image processing. Any point spread function effects, for example due to T2-blurring, that would decrease the effective resolution are not considered here."
{term}M0Estimate (metadata),"OPTIONAL, but REQUIRED when M0Type is defined as Estimate",number,"A single numerical whole-brain M0 value (referring to the M0 of blood), only if obtained externally (for example retrieved from CSF in a separate measurement). Must be a number greater than 0."
{term}BackgroundSuppressionNumberPulses (metadata),"OPTIONAL, RECOMMENDED if BackgroundSuppression is true",number,The number of background suppression pulses used. Note that this excludes any effect of background suppression pulses applied before the labeling. Must be a number greater than or equal to 0.
{term}BackgroundSuppressionPulseTime (metadata),"OPTIONAL, RECOMMENDED if BackgroundSuppression is true",array of numbers,"Array of numbers containing timing, in seconds, of the background suppression pulses with respect to the start of the labeling. In case of multi-PLD with different background suppression pulse times, only the pulse time of the first PLD should be defined."


(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:asl.json-file:pcasl-specific-metadata-fields)=
#### (P)CASL-specific metadata fields

These fields can only be used when `ArterialSpinLabelingType` is `"CASL"` or `"PCASL"`. See [Appendix XII - ASL](../99-appendices/12-arterial-spin-labeling.md#pcasl-sequence) for more information on the (P)CASL sequence and the Labeling Pulse fields.

In [22]:
field_info = {
   "LabelingDuration": "REQUIRED",
   "PCASLType": 'RECOMMENDED if `ArterialSpinLabelingType` is `"PCASL"`',
   "CASLType": 'RECOMMENDED if `ArterialSpinLabelingType` is `"CASL"`',
   "LabelingPulseAverageGradient": "RECOMMENDED",
   "LabelingPulseMaximumGradient": "RECOMMENDED",
   "LabelingPulseAverageB1": "RECOMMENDED",
   "LabelingPulseDuration": "RECOMMENDED",
   "LabelingPulseFlipAngle": "RECOMMENDED",
   "LabelingPulseInterval": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}LabelingDuration (metadata),REQUIRED,number or array of numbers,"Total duration of the labeling pulse train, in seconds, corresponding to the temporal width of the labeling bolus for ""PCASL"" or ""CASL"". In case all control-label volumes (or deltam or CBF) have the same LabelingDuration, a scalar must be specified. In case the control-label volumes (or deltam or cbf) have a different ""LabelingDuration"", an array of numbers must be specified, for which any m0scan in the timeseries has a ""LabelingDuration"" of zero. In case an array of numbers is provided, its length should be equal to the number of volumes specified in *_aslcontext.tsv. Corresponds to DICOM Tag 0018, 9258 ASL Pulse Train Duration."
{term}PCASLType (metadata),"RECOMMENDED if ArterialSpinLabelingType is ""PCASL""",string,"The type of gradient pulses used in the control condition. Must be one of: ""balanced"", ""unbalanced""."
{term}CASLType (metadata),"RECOMMENDED if ArterialSpinLabelingType is ""CASL""",string,"Describes if a separate coil is used for labeling. Must be one of: ""single-coil"", ""double-coil""."
{term}LabelingPulseAverageGradient (metadata),RECOMMENDED,number,"The average labeling gradient, in milliteslas per meter. Must be a number greater than 0."
{term}LabelingPulseMaximumGradient (metadata),RECOMMENDED,number,"The maximum amplitude of the gradient switched on during the application of the labeling RF pulse(s), in milliteslas per meter. Must be a number greater than 0."
{term}LabelingPulseAverageB1 (metadata),RECOMMENDED,number,"The average B1-field strength of the RF labeling pulses, in microteslas. As an alternative, ""LabelingPulseFlipAngle"" can be provided. Must be a number greater than 0."
{term}LabelingPulseDuration (metadata),RECOMMENDED,number,"Duration of the individual labeling pulses, in milliseconds. Must be a number greater than 0."
{term}LabelingPulseFlipAngle (metadata),RECOMMENDED,number,"The flip angle of a single labeling pulse, in degrees, which can be given as an alternative to ""LabelingPulseAverageB1"". Must be a number greater than 0 and less than or equal to 360."
{term}LabelingPulseInterval (metadata),RECOMMENDED,number,"Delay between the peaks of the individual labeling pulses, in milliseconds. Must be a number greater than 0."


(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:asl.json-file:pasl-specific-metadata-fields)=
#### PASL-specific metadata fields

These fields can only be used when `ArterialSpinLabelingType` is `PASL`. See [Appendix XII - ASL](../99-appendices/12-arterial-spin-labeling.md#pasl-sequence) for more information on the PASL sequence and the BolusCutOff fields.

In [23]:
field_info = {
   "BolusCutOffFlag": "REQUIRED",
   "PASLType": "RECOMMENDED",
   "LabelingSlabThickness": "RECOMMENDED",
   "BolusCutOffDelayTime": "OPTIONAL, REQUIRED if `BolusCutOffFlag` is `true`",
   "BolusCutOffTechnique": "OPTIONAL, REQUIRED if `BolusCutOffFlag` is `true`",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}BolusCutOffFlag (metadata),REQUIRED,boolean,"Boolean indicating if a bolus cut-off technique is used. Corresponds to DICOM Tag 0018, 925C ASL Bolus Cut-off Flag. Must be one of: ""true"", ""false""."
{term}PASLType (metadata),RECOMMENDED,string,"Type of the labeling pulse of the PASL labeling, for example ""FAIR"", ""EPISTAR"", or ""PICORE""."
{term}LabelingSlabThickness (metadata),RECOMMENDED,number,"Thickness of the labeling slab in millimeters. For non-selective FAIR a zero is entered. Corresponds to DICOM Tag 0018, 9254 ASL Slab Thickness. Must be a number greater than 0."
{term}BolusCutOffDelayTime (metadata),"OPTIONAL, REQUIRED if BolusCutOffFlag is true",number or array of numbers,"Duration between the end of the labeling and the start of the bolus cut-off saturation pulse(s), in seconds. This can be a number or array of numbers, of which the values must be non-negative and monotonically increasing, depending on the number of bolus cut-off saturation pulses. For Q2TIPS, only the values for the first and last bolus cut-off saturation pulses are provided. Based on DICOM Tag 0018, 925F ASL Bolus Cut-off Delay Time."
{term}BolusCutOffTechnique (metadata),"OPTIONAL, REQUIRED if BolusCutOffFlag is true",string,"Name of the technique used, for example ""Q2TIPS"", ""QUIPSS"", ""QUIPSSII"". Corresponds to DICOM Tag 0018, 925E ASL Bolus Cut-off Technique."


(magnetic-resonance-imaging:arterial-spin-labeling-perfusion-data:m0scan-metadata-fields)=
### `m0scan` metadata fields

Some common metadata fields are REQUIRED for the `*_m0scan.json`: `EchoTime`, `RepetitionTimePreparation`, and `FlipAngle` in case `LookLocker` is `true`.

In [24]:
field_info = {
   "IntendedFor": (
      "REQUIRED",
      "This is used to refer to the ASL time series for which the `*_m0scan.nii[.gz]` is intended."
   ),
   "AcquisitionVoxelSize": "RECOMMENDED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}IntendedFor (metadata),REQUIRED,string or array of strings,"The paths to files for which the associated file is intended to be used. Contains one or more filenames with paths relative to the participant subdirectory. Paths need to use forward slashes instead of backward slashes, regardless of operating system. This is used to refer to the ASL time series for which the *_m0scan.nii[.gz] is intended."
{term}AcquisitionVoxelSize (metadata),RECOMMENDED,array of numbers,"An array of numbers with a length of 3, in millimeters. This parameter denotes the original acquisition voxel size, excluding any inter-slice gaps and before any interpolation or resampling within reconstruction or image processing. Any point spread function effects, for example due to T2-blurring, that would decrease the effective resolution are not considered here."


The following table recapitulates the ASL field dependencies. If Source field (column 1) contains the Value specified in column 2, then the Requirements in column 4 are
imposed on the Dependent fields in column 3. See [Appendix XII](../99-appendices/12-arterial-spin-labeling.md#flowchart-based-on-dependency-table) for this information in the
form of flowcharts.

| **Source field**         | **Value**    | **Dependent field**  | **Requirements**                                 |
|--------------------------|--------------|----------------------|--------------------------------------------------|
| MRAcquisitionType        | 2D / 3D      | SliceTiming          | \[X\] / \[\]                                     |
| LookLocker               | true         | FlipAngle            | \[X\]                                            |
| ArterialSpinLabelingType | PCASL        | LabelingDuration     | \[X\]                                            |
| ArterialSpinLabelingType | PASL         | BolusCutOffFlag      | \[X\]                                            |
| BolusCutOffFlag          | true / false | BolusCutOffDelayTime | \[X\] / \[\]                                     |
| BolusCutOffFlag          | true / false | BolusCutOffTechnique | \[X\] / \[\]                                     |
| M0Type                   | Separate     | */perf/              | contains `*_m0scan.nii[.gz]` and `*_m0scan.json` |
| M0Type                   | Included     | *_aslcontext.tsv     | contains m0scan                                  |
| M0Type                   | Estimate     | M0Estimate           | \[X\]                                            |
| `*_aslcontext.tsv`       | cbf          | Units                | \[X\]                                            |

**Legend**

-   \[ X ] --> MUST be defined
-   \[   \] --> MUST NOT be defined

(magnetic-resonance-imaging:fieldmap-data)=
## Fieldmap data

Data acquired to correct for *B<sub>0</sub>* inhomogeneities can come in different forms.
The current version of this standard considers four different scenarios:

1.  [Phase-difference map](#case-1-phase-difference-map-and-at-least-one-magnitude-image)
1.  [Two phase maps](#case-2-two-phase-maps-and-two-magnitude-images)
1.  [Direct *field mapping*](#case-3-direct-field-mapping)
1.  ["*PEpolar*" fieldmaps](#case-4-multiple-phase-encoded-directions-pepolar)

These four different types of field mapping strategies can be encoded
using the following image types:

{{ MACROS___make_suffix_table(
      [
         "magnitude",
         "magnitude1",
         "magnitude2",
         "phase1",
         "phase2",
         "phasediff",
         "fieldmap",
         "epi",
      ]
   )
}}

Two OPTIONAL entities, following more general rules of the specification,
are allowed across all the four scenarios:

-   The OPTIONAL [`run-<index>`](../99-appendices/09-entities.md#run) key/value pair corresponds to a one-based index
    to distinguish multiple fieldmaps with the same parameters.

-   The OPTIONAL [`acq-<label>`](../99-appendices/09-entities.md#acq) key/value pair corresponds to a custom label
    the user may use to distinguish different set of parameters.

(magnetic-resonance-imaging:fieldmap-data:expressing-the-mr-protocol-intent-for-fieldmaps)=
### Expressing the MR protocol intent for fieldmaps

Fieldmaps are typically acquired with the purpose of correcting one or more EPI
scans under `dwi/`, `func/`, or `perf/` for distortions derived from *B<sub>0</sub>*
nonuniformity.

(magnetic-resonance-imaging:fieldmap-data:expressing-the-mr-protocol-intent-for-fieldmaps:using-b0fieldidentifier-metadata)=
#### Using `B0FieldIdentifier` metadata

The general purpose [`B0FieldIdentifier` MRI metadata](#echo-planar-imaging-and-b0-mapping)
is RECOMMENDED for the prescription of the *B<sub>0</sub>* field estimation intent of the
original acquisition protocol.
`B0FieldIdentifier` and `B0FieldSource` duplicate the capabilities of
the original `IntendedFor` approach (see below), while permitting more
complex use cases.
It is RECOMMENDED to use both approaches to maintain compatibility with
tools that support older datasets.

(magnetic-resonance-imaging:fieldmap-data:expressing-the-mr-protocol-intent-for-fieldmaps:using-intendedfor-metadata)=
#### Using `IntendedFor` metadata

Fieldmap data MAY be linked to the specific scan(s) it was acquired for by
filling the `IntendedFor` field in the corresponding JSON file.

In [25]:
field_info = {
   "IntendedFor": (
      "OPTIONAL",
      "This field is OPTIONAL, and in case the fieldmaps do not correspond "
      "to any particular scans, it does not have to be filled.",
   ),
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}IntendedFor (metadata),OPTIONAL,string or array of strings,"The paths to files for which the associated file is intended to be used. Contains one or more filenames with paths relative to the participant subdirectory. Paths need to use forward slashes instead of backward slashes, regardless of operating system. This field is OPTIONAL, and in case the fieldmaps do not correspond to any particular scans, it does not have to be filled."


For example:

```JSON
{
   "IntendedFor": [
        "ses-pre/func/sub-01_ses-pre_task-motor_run-1_bold.nii.gz",
        "ses-pre/func/sub-01_ses-pre_task-motor_run-2_bold.nii.gz"
    ]
}
```

(magnetic-resonance-imaging:fieldmap-data:types-of-fieldmaps)=
### Types of fieldmaps

(magnetic-resonance-imaging:fieldmap-data:types-of-fieldmaps:case-1)=
#### Case 1: Phase-difference map and at least one magnitude image

[Example datasets](https://github.com/bids-standard/bids-examples)
containing that type of fieldmap can be found here:

-   [`7t_trt`](https://github.com/bids-standard/bids-examples/tree/master/7t_trt)
-   [`genetics_ukbb`](https://github.com/bids-standard/bids-examples/tree/master/genetics_ukbb)
-   [`ds000117`](https://github.com/bids-standard/bids-examples/tree/master/ds000117)

{{ MACROS___make_filename_template(datatypes=["fmap"], suffixes=["phasediff", "magnitude1", "magnitude2"]) }}

where
the REQUIRED `_phasediff` image corresponds to the phase-drift map between echo times,
the REQUIRED `_magnitude1` image corresponds to the shorter echo time, and
the OPTIONAL `_magnitude2` image to the longer echo time.

Required fields:

In [26]:
field_info = {
   "EchoTime1": "REQUIRED",
   "EchoTime2": "REQUIRED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}EchoTime1 (metadata),REQUIRED,number,The time (in seconds) when the first (shorter) echo occurs. Must be a number greater than 0.
{term}EchoTime2 (metadata),REQUIRED,number,The time (in seconds) when the second (longer) echo occurs. Must be a number greater than 0.


In this particular case, the sidecar JSON file
`sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phasediff.json`
MUST define the time of two echos used to map the phase and finally calculate
the phase-difference map.
For example:

```JSON
{
   "EchoTime1": 0.00600,
   "EchoTime2": 0.00746,
   "B0FieldIdentifier": "phasediff_fmap0"
}
```

(magnetic-resonance-imaging:fieldmap-data:types-of-fieldmaps:case-2)=
#### Case 2: Two phase maps and two magnitude images

Similar to case 1, but instead of a precomputed phase-difference map, two
separate phase images and two magnitude images corresponding to first and
second echos are available.

{{ MACROS___make_filename_template(datatypes=["fmap"], suffixes=["phase1", "phase2", "magnitude1", "magnitude2"]) }}

Required fields:

In [27]:
field_info = {
   "EchoTime__fmap": "REQUIRED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}EchoTime (metadata),REQUIRED,number,The time (in seconds) when the echo corresponding to this map was acquired. Must be a number greater than 0.


Each phase map has a corresponding sidecar JSON file to specify its corresponding `EchoTime`.
For example, `sub-<label>[_ses-<label>][_acq-<label>][_run-<index>]_phase2.json` may read:

```JSON
{
   "EchoTime": 0.00746,
   "B0FieldIdentifier": "phases_fmap0"
}
```

(magnetic-resonance-imaging:fieldmap-data:types-of-fieldmaps:case-3)=
#### Case 3: Direct *field mapping*
In some cases (for example GE), the scanner software will directly reconstruct a
*B<sub>0</sub>* field map along with a magnitude image used for anatomical reference.

{{ MACROS___make_filename_template(datatypes=["fmap"], suffixes=["fieldmap", "magnitude"]) }}

Required fields:

In [28]:
field_info = {
   "Units": (
      "REQUIRED",
      'Fieldmaps must be in units of Hertz (`"Hz"`), '
      'radians per second (`"rad/s"`), or Tesla (`"T"`).',
   ),
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}Units (metadata),REQUIRED,string,"Measurement units for the associated file. SI units in CMIXF formatting are RECOMMENDED (see Units). Fieldmaps must be in units of Hertz (""Hz""), radians per second (""rad/s""), or Tesla (""T"")."


For example:

```JSON
{
   "Units": "rad/s",
   "IntendedFor": "func/sub-01_task-motor_bold.nii.gz",
   "B0FieldIdentifier": "b0map_fmap0"
}
```

See [Using `IntendedFor` metadata](#using-intendedfor-metadata)
for details on the `IntendedFor` field.

(magnetic-resonance-imaging:fieldmap-data:types-of-fieldmaps:case-4)=
#### Case 4: Multiple phase encoded directions ("pepolar")

An [example dataset](https://github.com/bids-standard/bids-examples)
containing that type of fieldmap can be found here:

-   [`ieeg_visual_multimodal`](https://github.com/bids-standard/bids-examples/tree/master/ieeg_visual_multimodal)

The phase-encoding polarity (PEpolar) technique combines two or more Spin Echo
EPI scans with different phase encoding directions to estimate the distortion
map corresponding to the nonuniformities of the *B<sub>0</sub>* field.
These `*_epi.nii[.gz]` - or `*_m0scan.nii[.gz]` for arterial spin labeling perfusion data - files can be 3D or 4D --
in the latter case, all timepoints share the same scanning parameters.
Examples of software tools using these kinds of images are FSL TOPUP,
AFNI `3dqwarp`, and SPM.

{{ MACROS___make_filename_template(datatypes=["fmap"], suffixes=["epi"]) }}

The [`dir-<label>`](../99-appendices/09-entities.md#dir) entity is REQUIRED
for these files.
This key-value pair MUST be used in addition to
the REQUIRED `PhaseEncodingDirection` metadata field
(see [File name structure](../02-common-principles.md#file-name-structure)).

Required fields:

In [29]:
field_info = {
   "PhaseEncodingDirection": "REQUIRED",
   "TotalReadoutTime": "REQUIRED",
}

table = render.make_metadata_table(schema_obj, field_info)
display(HTML(markdown(table.to_markdown(), extensions=['tables'])))

Key name,Requirement Level,Data type,Description
{term}PhaseEncodingDirection (metadata),REQUIRED,string,"The letters i, j, k correspond to the first, second and third axis of the data in the NIFTI file. The polarity of the phase encoding is assumed to go from zero index to maximum index unless - sign is present (then the order is reversed - starting from the highest index instead of zero). PhaseEncodingDirection is defined as the direction along which phase is was modulated which may result in visible distortions. Note that this is not the same as the DICOM term InPlanePhaseEncodingDirection which can have ROW or COL values. Must be one of: ""i"", ""j"", ""k"", ""i-"", ""j-"", ""k-""."
{term}TotalReadoutTime (metadata),REQUIRED,number,"This is actually the ""effective"" total readout time, defined as the readout duration, specified in seconds, that would have generated data with the given level of distortion. It is NOT the actual, physical duration of the readout train. If ""EffectiveEchoSpacing"" has been properly computed, it is just EffectiveEchoSpacing * (ReconMatrixPE - 1)."


For example:

```JSON
{
   "PhaseEncodingDirection": "j-",
   "TotalReadoutTime": 0.095,
   "IntendedFor": "func/sub-01_task-motor_bold.nii.gz",
   "B0FieldIdentifier": "pepolar_fmap0"
}
```

See [Using `IntendedFor` metadata](#using-intendedfor-metadata)
for details on the `IntendedFor` field.

As for other EPI sequences, these field mapping sequences may have any of the
[in-plane spatial encoding](#in-plane-spatial-encoding) metadata keys.
However, please note that `PhaseEncodingDirection` and `TotalReadoutTime` keys
are REQUIRED for these field mapping sequences.

<!-- Link Definitions -->